# Imports

In [2]:
!pip install python-dotenv

In [3]:
import requests
from kafka import KafkaProducer
import json
from datetime import datetime, timedelta
from dotenv import load_dotenv
import os
import time

In [4]:
# Charger les variables d'environnement depuis le fichier .env copié dans le conteneur
load_dotenv('/home/jovyan/.env')

# Configuration de l'API OpenSky
OPENSKY_ALL_FLIGHTS_URL = "https://opensky-network.org/api/flights/all?begin={}&end={}"
USERNAME = os.environ.get('OPENSKY_USERNAME')
PASSWORD = os.environ.get('OPENSKY_PASSWORD')

# Fonction pour envoyer les données OpenSky à Kafka

In [5]:

def send_opensky_to_kafka(topic, url, username, password):
    # Kafka configuration
    kafka_config = {
        'bootstrap_servers': 'kafka1:9092',  # Update with your Kafka broker
    }

    # Initialize Kafka Producer
    producer = KafkaProducer(
        bootstrap_servers=kafka_config['bootstrap_servers'],
        value_serializer=lambda v: json.dumps(v).encode('utf-8')
    )

    # Fetch data from OpenSky
    response = requests.get(url, auth=(username, password))

    if response.status_code == 200:
        data = response.json()
        if isinstance(data, list) and len(data) > 0 and isinstance(data[0], dict):
            flights = [item for item in data if 'icao24' in item]
        else:
            flights = []


        # Publish each state to Kafka
        for flight in flights:
            producer.send(topic, value=flight)
            #print(f"Sent: {flight}")

        # Ensure all messages are sent
        producer.flush()
        print(f"Sent {len(flights)} records.")
    else:
        print(f"Failed to fetch data: {response.status_code}")

In [6]:
flights = {
    "icao24": "icao24",
    "firstSeen": "firstSeen",
    "estDepartureAirport": "estDepartureAirport",
    "lastSeen": "lastSeen",
    "estArrivalAirport": "estArrivalAirport",
    "estDepartureAirportHorizDistance": "estDepartureAirportHorizDistance",
    "estDepartureAirportVertDistance": "estDepartureAirportVertDistance",
    "estArrivalAirportHorizDistance": "estArrivalAirportHorizDistance",
    "estArrivalAirportVertDistance": "estArrivalAirportVertDistance",
    "departureAirportCandidatesCount": "departureAirportCandidatesCount",
    "arrivalAirportCandidatesCount": "arrivalAirportCandidatesCount",
}

# Durée de récupération (minutes)
step = 60

# date initiale
initial_date_str = "2025-02-19 10:00:00"

date_format = "%Y-%m-%d %H:%M:%S"
initial_date = datetime.strptime(initial_date_str, date_format)
end_date = initial_date + timedelta(minutes=step)


start = int(initial_date.timestamp())
end = int(end_date.timestamp())




In [7]:
# Send OpenSky data to Kafka

for i in range(20):
    print(f"{start}-{end}")
    send_opensky_to_kafka("opensky-flights", OPENSKY_ALL_FLIGHTS_URL.format(start, end), USERNAME, PASSWORD)
    time.sleep(5)
    initial_date = end_date
    end_date = initial_date + timedelta(minutes=step)
    start = int(initial_date.timestamp())
    end = int(end_date.timestamp())

1739959200-1739962800
Sent 677 records.
1739962800-1739966400
Sent 708 records.
1739966400-1739970000
Sent 797 records.
1739970000-1739973600
Sent 946 records.
1739973600-1739977200
Sent 1000 records.
1739977200-1739980800
Sent 1179 records.
1739980800-1739984400
Sent 1107 records.
1739984400-1739988000
Sent 1064 records.
1739988000-1739991600
Sent 1009 records.
1739991600-1739995200
Sent 989 records.
1739995200-1739998800
Sent 1043 records.
1739998800-1740002400
Sent 908 records.
1740002400-1740006000
Sent 1722 records.
1740006000-1740009600
Failed to fetch data: 404
1740009600-1740013200
Sent 540 records.
1740013200-1740016800
Sent 442 records.
1740016800-1740020400
Sent 210 records.
1740020400-1740024000
Sent 31 records.
1740024000-1740027600
Sent 646 records.
1740027600-1740031200
Sent 764 records.
